## สคลิปทดสอบนี้ ใช้สำหรับ ทดสอบผ่านข้อความให้กับ LLM แล้ว
## ใช้คำถาม ถามตามข้อความที่ส่งให้

In [1]:
from constants import *
from gpt4allV1_3 import pyllmodel
from langchain.prompts import PromptTemplate

model_path = f'{LLM_MODEL_PATH}\\{LLM_MODEL_NAME}'
llm = pyllmodel.LLModel()
llm.load_model(model_path=model_path)

d:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
query_str = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer, answer short questions without explanation, make up an answer of no more than 3 sentences, there must be no test cases out there in the answer.

    Users can deposit cash amounts between 9.00-17.00 If the user�s age is between 12- and 15 years old the fee amount charged is 0.00 baht.
    Feature: Deposit cash balance Conditions: Users can�t deposit cash amounts over 20,000 baht per transaction.
    Users can�t deposit cash amounts over 500,000 baht per day.
    If the user�s age is over 15 years old the fee amount charged is 15 baht per transaction.
    1. Test Case of Unit Testing 2. Test Case of Integration Testing 3. Test Scenario of End-to-End Testing

    Question: Can I deposit more than 20,000 baht in one transaction?
    Helpful Answer:
"""
# 

llm.prompt_model(
     prompt=query_str,
        streaming=True,
        reset_context=True,

        # ค่าอยู่ระหว่าง 0 - 1 เช่น 0.1, 0.2, 0.3, 0.4, ..., 1
        # เป็นค่าความคิดสร้างสรรค์ ในการตอบคำถามของ LLM
        # นัยสำคัญของ Fine-tune:
        #   ค่า temp จะมีนัยสำคัญในการนำข้อความ ที่ผ่านให้กับ LLM ด้วย Prompt โดย LLM จะนำ Prompt นั้นมาวิเคราะห์ข้อความที่อยู่ภายใน Prompt เพื่อให้ได้ซึ่งคำตอบ
        # ตัวอย่างเช่น:
        #   Feature: Deposit cash balance Conditions: Users can't deposit cash amounts over 20,000 baht per transaction.\n 
        # Users can’t deposit cash amounts over 500,000 baht per day.\n
        # Users can deposit cash amounts between 9.00-17.00 If the user’s age is between 12- and 15 years old the fee amount charged is 0.00 baht.\n
        #
        #  Question: Can I deposit more than 500,000 baht per day?
        #  ถ้ากำหนด temp = 0.3 คำตอบที่ได้จะเป็น Yes เสมอ
        #  ถ้ากำหนด temp = 0.5 คำตอบที่ได้จะเป็น No แต่รูปแบบประโยค ยังไม่สมบูรณ์
        #  ถ้ากำหนด temp = 0.6 คำตอบที่ได้จะเป็น No แต่รูปแบบประโยค ยังไม่สมบูรณ์ แต่ทิศทางดีกว่า temp = 0.5
        #  ถ้ากำหนด temp = 0.7 คำตอบที่ได้จะเป็น No แต่รูปแบบประโยค เริ่มสมบูรณ์ขึ้น แต่ยังมีบางส่วนที่ต้องแก้ไข
        #  ถ้ากำหนด temp = 0.8 คำตอบที่ได้จะเป็น No รูปแบบประโยคเริ่มถูกต้อง และมีทิศทางทีดี (แนะนำให้ใช้ค่าประมาณนี้)
        #
        # คำแนะนำและบริษทในการปรับค่า:
        #   A. หากนำ LLM นี้ไปใช้งานในเอกสารที่เป็น ลักษณะ Q & A ซึ่งมีคำถามและคำตอบ ในเนื้อหาอยู่แล้ว แนะนำให้ใช้ค่าที่ 0.3 ก็เพียงพอแล้ว
        #   B. หากนำ LLM นี้ไปใช้งานในเอกสารที่มีเนื้อหา ที่ต้องการให้ LLM นำเนื้อหานั้นมาวิเคราะห์ข้อความ เพื่อสังเคราะห์ออกมาเป็นคำตอบให้ปรับค่าเป็น 0.8 จะถือเป็นค่าที่เหมาะสม
        temp=0.3,


        # จำนวน Token (ชุดของข้อความที่มีความเป็นไปได้ ที่ใกล้งเคียงกับคำถาม) ที่บอกให้ LLM หยิบมาใช้ในการสร้างประโยคคำตอบ
        # ซึ่งรายการ Token ที่ได้จะถูกจัดลำดับ (sort) ตามความน่าจะเป็นที่ มากสุดไปน้อย สุด
        # ค่านี้จะมีผลต่อ ความยาวของประโยคคำตอบ
        # กำหนดค่านี้เป็น 1 จะได้ประโยคคำตอบที่สั้นและกระชับ
        top_k=1, 


        # ผลรวม % จากจำนวนทั้งหมดของ top_k (คือ 20 รายการ)
        # จะต้องได้ >= top_p ถึงจะออกมาเป็นคำตอบ
        # หลักๆ แล้วจะใช้ในการ กรองข้อมูล Token ที่มีความน่าจะเป็น ของคำตอบน้อย ออกไป
        top_p=0.9,

        # จำนวนคำ (Token) สูงสุด ที่นำมาสร้างเป็นรูปประโยคคำตอบ สูงสุดไม่เกินจำนวนกี่คำ
        # หากเกินจำนวนคำที่กำหนดแล้ว ถึงแม้ประโยคจะไม่สมบูรณ์ ก็จะหยุดสร้างคำตอบทันที (Stop Generate Text)
        n_predict = 128,


        # จำนวน Prediction (การคาดการณ์/การทำนาย คำถาม หรือ จำนวนคำถาม) ที่จะเกิดขึ้นระหว่างการสร้างคำตอบ (Generate text)
        # ส่วนใหญ่จะเกิดในกรณีที่ Q & A
        # เช่น
        # Q: Is it possible to buy and sell shares when the Share Status is CLOSEONLY? \
        # A: No, it is not possible to buy and sell shares when the Share Status is CLOSEONLY.
        # What are the possible values for the CanBuy and CanSell Account Status options?
        #
        # เมื่อ LLM สร้างคำตอบ สิ่งที่ได้จะเป็นส่วนของ A (Answer) และมีคำถามติดมาด้วย What are ...
        # หากกำหนดค่า n_batch 2,3,4,5, ... LLM ก็จะค้นหาคำตอบเพิ่มเติมจากคำถามที่พบ
        n_batch=1500,

        
        

        #repeat_penalty = 0,
        #repeat_last_n = 0,
)

No, you can't deposit more than 20,000 baht in one transaction.

"No, you can't deposit more than 20,000 baht in one transaction."